# SIB KAGGLE PROJECT

In [1]:
#Imports
from utils.func import *
from utils.kmers import KMer_prot
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
#First lets see the data!
#Train dataframe
train = pd.read_csv("data/train.csv",index_col="seq_id")
#Test dataframe
test = pd.read_csv("data/test.csv",index_col="seq_id")
print(f"Train data is divided in {train.shape[0]} lines and {train.shape[1]} col")
print(f"Test data is divided in {test.shape[0]} lines and {test.shape[1]} col")
print(f"Labels: {[labels for labels in train.columns]}")
print("Remove data_source")
train= train.drop(columns="data_source")
test= test.drop(columns="data_source")
print(f"Train data is divided in {train.shape[0]} lines and {train.shape[1]} col")
print(f"Test data is divided in {test.shape[0]} lines and {test.shape[1]} col")
print("We want to predict tm values for test data")

Train data is divided in 31390 lines and 4 col
Test data is divided in 2413 lines and 3 col
Labels: ['protein_sequence', 'pH', 'data_source', 'tm']
Remove data_source
Train data is divided in 31390 lines and 3 col
Test data is divided in 2413 lines and 2 col
We want to predict tm values for test data


### The data possesses swapped values between the pH and tm columns, then, "swap_ph_tm" was built to fix this issue.

In [3]:
update_train= pd.read_csv("data/train_updates_20220929.csv",index_col="seq_id")
train= swap_ph_tm(train,update_train)


In [4]:
train

,protein_sequence,pH,tm
seq_id,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5
...,...,...,...
31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8
31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2
31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6


In [5]:
print(train.isnull().sum().sort_values(ascending=False))
print(test.isnull().sum().sort_values(ascending=False))
#There are some missing values in train
#Data_source values are not that important

pH                  286
protein_sequence      0
tm                    0
dtype: int64
protein_sequence    0
pH                  0
dtype: int64


In [6]:
missing_data= train[train["pH"].isnull()]
missing_data

,protein_sequence,pH,tm
seq_id,,,
484,ARINTVRGPITISEAGFTLTHEHICGSSAGFLRAWPEFFGSRKALA...,NaN,84.0
6215,MASHKLLVTPLKALLKPLSIPNQLLLGPGPSNLPPRIMAAGGLQMI...,NaN,73.8
6216,MASHKLLVTPPKALLKPLSIPNQLLLGPGPSNLPPRIMAAGGLQMI...,NaN,85.5
6217,MASHKLLVTPPKALLKPLSIPNQLLLGPGPSNLPPRIMAAGGLQMI...,NaN,67.1
6218,MASHKLLVTPPKALLKPLSIPNQLLLGPGPSNLPPRIMAAGGLQMI...,NaN,83.2
...,...,...,...
28753,MVLKQRANYLGFLIVFFTAFLVEAVPIKRQSNSTVDSLPPLIPSRT...,NaN,58.9
28754,MVLKQRANYLGFLIVFFTAFLVEAVPIKRQSNSTVDSLPPLIPSRT...,NaN,59.4
28755,MVLKQRANYLGFLIVFFTAFLVEAVPIKRQSNSTVDSLPPLIPSRT...,NaN,57.8


In [7]:
train= train.drop((missing_data).index)
train
#Podemos remover também a data_source? Não deve de trazer nada de relevante para a analise dos dados

,protein_sequence,pH,tm
seq_id,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5
...,...,...,...
31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8
31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2
31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6


In [8]:
print(f"Train data is divided in {train.shape[0]} lines and {train.shape[1]} col")
print(f"Test data is divided in {test.shape[0]} lines and {test.shape[1]} col")
print(f"Labels: {[labels for labels in train.columns]}")


Train data is divided in 28695 lines and 3 col
Test data is divided in 2413 lines and 2 col
Labels: ['protein_sequence', 'pH', 'tm']


# Supervised Machine Learning

## KNNClassifier Model

#### !!!! Transform Sequences in Values !!!!

In [9]:
sequences = train.loc[:, "protein_sequence"].values
kmer = KMer_prot(2)
new_data = kmer.fit_transform(sequences[:10])
for lista in new_data:
  for value in lista:
    if value != 0:
      print(True)

#### Verificação da Média e Desvio Padrão dos dados de treino:

In [10]:
quantitative_data_train = train.loc[:, "pH":"tm"].values
pre_processed_train = preprocessing.scale(quantitative_data_train)

print("Media global: ", pre_processed_train.mean())
print("Desvio padrao global: ", pre_processed_train.std())
print("\nVerificar se a média e o desvio padrão estão próximos dos valores 0 e 1, respetivamente.")
print("\tMédia:", ((pre_processed_train.mean(axis=0) < 0.000001) & (pre_processed_train.mean(axis=0) > -0.000001)).all())
print("\tDesvio Padrão:", ((pre_processed_train.std(axis=0) < 1.000001) & (pre_processed_train.std(axis=0) > 0.999999)).all())

Media global:  2.2496186633143443e-16
Desvio padrao global:  1.0

Verificar se a média e o desvio padrão estão próximos dos valores 0 e 1, respetivamente.
	Média: True
	Desvio Padrão: True


In [11]:
from sklearn.neighbors import KNeighborsClassifier

# Divide the train data to input and output
train_input = train.loc[:, "protein_sequence":"pH"].values
train_output = train.loc[:, "tm"].values

# knn = KNeighborsClassifier()
# knn.fit(train_input, train_output)
# test_prevision = knn.predict(test)
# print(test_prevision)

NameError: name 'alignments' is not defined